# iPython Nutrition Notebook

This notebook was created to make it easier to understand how several behavioral variables affect your body's composition. Please add or remove from it what you like, as it was only ever intended to aid the otherwise obnoxiously tedious activity that is macro tracking.

## Initial setup

In [ ]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

#Start MongoDB Client
client = MongoClient()
db = client.primer
coll = db.dataset

#FOODS

#Simple Foods
empty = np.array([0.,0.,0.,0.])
Egg = np.array([78., 5., 0.6, 6.])
Avocado_whole = np.array([234., 21., 12., 2.9])
Sour_Cream_1tbsp = np.array([23., 2.4, 0.3, 0.2])
Spinach = np.array([7., 0.1, 1.1, 0.9])
Chicken_1oz = np.array([32.5, 0.75, 0, 6.5])
Peanut_Butter_2tbsp = np.array([200., 16., 6., 8.])
Refried_Beans_1serving = np.array([195, 0.89, 33.7, 10.64])
Trail_Mix_hfcup = np.array([360, 24., 31.2, 12.])
Protein_Powder_1scoop = np.array([140., 2., 6., 24.])
Oatmeal_hfcup = np.array([150.0, 3.0, 27.0, 5.0])
Sweet_Potato_1oz = np.array([86./3.5274, 0.1/3.5274, 5.6698985088, 1.6/3.5274])
White_Rice_1cp = np.array([206, 0.4, 45.0, 4.3])
One_Percent_Milk_1cup = np.array([130, 2.5, 16., 11.])
Two_Percent_Milk_1cup = np.array([130., 5., 13., 10.])
Cottage_Cheese_hfcup = np.array([80.0, 0.0, 6.0, 13.0])

#Protein
Amplified_Mass_1scoop = np.array([187.5, 1.5, 31, 12.5])
Protein_Powder_MochaCappuccino = np.array([120, 1.5, 3., 24.])
Protein_Powder_DoubleChocolate = np.array([120, 1., 3., 24.])

#Mealy Foods
chicken_serving_small = 8*Chicken_1oz
chicken_serving_large = 10*Chicken_1oz
halfAnAvocado = np.multiply(0.5, Avocado_whole)
quarterAnAvocado = np.multiply(0.25, Avocado_whole)
omelette = 6*Egg + halfAnAvocado + 6*Sour_Cream_1tbsp + Spinach
omeletteLessFat = 6*Egg + quarterAnAvocado + Spinach
oatmeal = 2*Oatmeal_hfcup
halfMass = 2*Amplified_Mass_1scoop
halfMassWithMilk = halfMass + One_Percent_Milk_1cup

## Enums
#Macronutrients
Fat, Carbohydrates, Protein = range(3)
#Gender
Male, Female = range(2)
#Activity Level
Sedentary, Light, Moderate, VeryActive = range(4)

    
#utility functions
def lbsToKG(weightInLbs):
    return weightInLbs*0.453592

def inToCm(heightInIn):
    return heightInIn*2.54

def calculateTotalCalories(weightInLbs, heightInIn, age, gender, activityLevel):
    restingEnergyExpenditure = 0.
    if gender == Male:
        restingEnergyExpenditure = 10*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5*age + 5.
    elif gender == Female:
        restingEnergyExpenditure = 10.*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5.*age - 161.
        
    totalDailyEnergyExpenditure = ([1.2, 1.375, 1.55, 1.725][activityLevel]) * restingEnergyExpenditure
    
    return totalDailyEnergyExpenditure

def calculateGramsOfMacro(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = 0.0
    gramsCarbs = 0.0
    gramsFat = 0.0
    
    if activityLevel <= 1:
        gramsProtein = weightInLbs*0.825
    else:
        gramsProtein = weightInLbs
    gramsFat = (targetCalories * 0.25)/9.0
    gramsCarbs = (targetCalories - (gramsFat*9.0 + gramsProtein*4.0))/4.0 
    
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    return returnArray[eval(macroString)]

def calculateGramsOfMacroLeft(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = calculateGramsOfMacro(targetCalories, currentMacros, "Protein", weightInLbs, activityLevel)
    gramsCarbs = calculateGramsOfMacro(targetCalories, currentMacros, "Carbohydrates", weightInLbs, activityLevel)
    gramsFat = calculateGramsOfMacro(targetCalories, currentMacros, "Fat", weightInLbs, activityLevel)
    
    gramsProtein = gramsProtein - currentMacros[3]
    gramsCarbs = gramsCarbs - currentMacros[2]  
    gramsFat = gramsFat - currentMacros[1]
    
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    return returnArray[eval(macroString)]

def calculateServingsOfFoodLeftByMacro(targetMacroAmountInGrams, currentMacros, macroString, food):
    gramsLeft = 0.0
    servings = 0.0
    
    gramsLeft = targetMacroAmountInGrams - currentMacros[eval(macroString)+1]
    servings = gramsLeft/eval(food)[eval(macroString)+1]
    
    if servings < 0:
        print "Over " + `macroString` + " budget by " + `-gramsLeft` + " grams."
        print "Equivalent to " + `-gramsLeft/eval(food)[eval(macroString)+1]` + " servings of " + `food`

    return servings
    

## Tracking

In [ ]:
#Beginning-of-day tracking
weight = 160
bodyfatpercentage = 12
sleep = {"hours": 9, "minutes": 2}

#Meal tracking
firstMeal = Protein_Powder_DoubleChocolate + omeletteLessFat
print("First Meal Calories: " + `firstMeal[0]`)
print("First Meal Fat: " + `firstMeal[1]`)
print("First Meal Carbs: " + `firstMeal[2]`)
print("First Meal Protein: " + `firstMeal[3]` + "\n")

secondMeal = 3*oatmeal
print("Second Meal Calories: " + `secondMeal[0]`)
print("Second Meal Fat: " + `secondMeal[1]`)
print("Second Meal Carbs: " + `secondMeal[2]`)
print("Second Meal Protein: " + `secondMeal[3]` + "\n")

thirdMeal = 42.8*Sweet_Potato_1oz 
print("Third Meal Calories: " + `thirdMeal[0]`)
print("Third Meal Fat: " + `thirdMeal[1]`)
print("Third Meal Carbs: " + `thirdMeal[2]`)
print("Third Meal Protein: " + `thirdMeal[3]` + "\n")

fourthMeal = 8.2*Chicken_1oz
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

total = firstMeal + secondMeal + thirdMeal + fourthMeal
print("Total Calories: " + `total[0]`)
print("Total Fat: " + `total[1]`)
print("Total Carbs: " + `total[2]`)
print("Total Protein: " + `total[3]`)


## Useful Calculations
The following cell may be useful for doing common macronutrient calculations.

In [ ]:
macronutrientName = "Carbohydrates"
foodName = "Sweet_Potato_1oz"
weight = 162
heightInIn = 72
gender = Male
age = 22
activityLevel = Moderate

maintenanceCalories = calculateTotalCalories(weight, heightInIn, age, gender, activityLevel)
gramsOfMacroLeftToConsume = calculateGramsOfMacroLeft(maintenanceCalories, total, macronutrientName, weight, activityLevel)
gramsOfMacrosTotal = calculateGramsOfMacro(maintenanceCalories, total, macronutrientName, weight, activityLevel)

print "Daily calories for maintaining current weight: " + `calculateTotalCalories(weight, heightInIn, age, gender, activityLevel)`
print "Grams of " + `macronutrientName` + " left: " + `gramsOfMacroLeftToConsume`
print "Additional servings of " + `foodName` + " to meet daily " + `macronutrientName` + " intake: " + `calculateServingsOfFoodLeftByMacro(gramsOfMacrosTotal, total, macronutrientName, foodName)`

## Data retrieval
Use this cell to view all data collected over a 'daysShown'-day period.

In [ ]:
#lists to store tracked variables
daysList = []
caloriesList = []
fatList = []
carbsList = []
proteinList = []
sleepList = []
weightList = []

#setup for mongo search
daysShown = 9
earliestDate = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - daysShown
cursor = db.macros.find( { 'date': { '$gt': earliestDate } } )

#add each entry's fields to the respective lists
for document in cursor:
    daysList.append(document['date'])
    caloriesList.append(document['macros']['calories'])
    fatList.append(document['macros']['fat'])
    carbsList.append(document['macros']['carbohydrates'])
    proteinList.append(document['macros']['protein'])
    weightList.append(document['weight'])
    sleepList.append((document['sleep']['hours'], document['sleep']['minutes']))
    print(document)
    
combinedSleepList = []
for (hours, minutes) in sleepList:
    combinedSleepList.append(hours+minutes/60.)
    

## Graphing.

In [ ]:
today = dt.date.today()


f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(daysList, caloriesList, color='y')
ax2.plot(daysList, weightList, color='b')
ax3.plot(daysList, carbsList, color='r')
ax4.plot(daysList, combinedSleepList, color='g')

ax1.set_ylim([min(caloriesList), max(caloriesList)*1.5])
ax2.set_ylim([min(weightList), max(weightList)*1.1])
ax3.set_ylim([min(carbsList), max(carbsList)*1.5])
ax4.set_ylim([min(combinedSleepList), max(combinedSleepList)*1.3])

ax1.set_title('Calories')
ax2.set_title('Weight')
ax3.set_title('Carbohydrates')
ax4.set_title('Sleep')


plt.savefig(`today.month` + "-" + `today.day` + "-" + `today.year` + ".png")


## Database Insertion
Use this cell to insert today's data into your MongoDB database

In [ ]:
daysSinceEpoch = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - 1

result = db.macros.insert_one(
    {
        "date": daysSinceEpoch,
        "macros": {
            "calories": total[0],
            "fat": total[1],
            "carbohydrates": total[2],
            "protein": total[3]
        },
        "weight": weight,
        "bodyfat%": bodyfatpercentage,
        "sleep": sleep
    }
)

In [ ]:
#db.macros.delete_one({'date': 16857})